## 다음 문제를 풀이하시오.

diabetes.csv 파일을 대상으로 다음 문제를 풀이하시오.

### Q1. 당뇨 여부인 Outcome과 임신 횟수인 Pregnancies을 제외한 수치형 변수는  결측치가 0으로 기록이 되어 있다. 모든 변수의 결측치 개수를 더한 개수를 기술하시오.
(정답 예시: 123)

In [48]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/diabetes.csv")

# Way 1
df_sub = df.drop(["Pregnancies", "Outcome"], axis=1)
print((~df_sub[df_sub==0].isnull()).sum().sum())

# Way 2
df_q1 = df.drop(["Pregnancies", "Outcome"], axis=1).copy()
df_q1
print((~df_q1[df_q1==0].isna()).sum().sum())
print((df_q1[df_q1==0].notna()).sum().sum())
print((df_q1[df_q1.isin([0])].notna()).sum().sum())


#  Way 3
print(df_q1[df_q1==0].count().sum())
print(df_q1[df_q1.isin([0])].count().sum())

# Way4
arr = []
answer = 0
for col in df_q1.columns:
    arr.append((df_q1[col] == 0).sum())
    answer += (df_q1[col] == 0).sum()
print(answer)


# Way5 Lambda
print(df_q1.apply(lambda x: x.isin([0]).sum()).sum())
print(df_q1.apply(lambda x: x[x == 0].notna().sum()).sum())


652
652
652
652
652
652
652
652
652


### Q2. Outcome 변수를 제외한 변수간 상관분석을 실시하고 상관계수가 가장 높은 조합의 변수명과 상관계수를 기술하시오.
 - 변수명은 알파벳 순서대로 기술한다.
 - 상관계수는 반올림하여 소수점 셋째자리까지 기술한다.
 
(정답 예시: aaa, bbb, 0.123)

In [2]:
corr = df.drop("Outcome", axis=1).corr()

## idxmax Mistake!!!!!
# corr_max = pd.concat([corr[corr != 1].max(), corr[corr != 1].idxmax()], axis=1)
# # corr_max[0].idxmax()
# corr_max.max()

# arr = [corr_max[0].idxmax(), corr_max.max()[1]]
# arr.sort()

# print(f"{arr[0]}, {arr[1]}, {round(corr_max.max()[0], 3)}")

corr_max_2 = corr.apply(lambda x: x[x != 1].max())
corr_max_2[corr_max_2 == corr_max_2.max()]

Pregnancies    0.544341
Age            0.544341
dtype: float64

### Q3. 임신 경험 여부와 BMI간의 관계를 보고자 한다. 이때 적절한 검정방법을 사용하고 해당 검정의 검정통계량의 절대값을 반올림하여 소수점 셋째 자리 까지 표기하시오.
(정답 예시: 1.234)

In [3]:
import statsmodels.formula.api as smf
from scipy.stats import ttest_ind

df_q3 = df.copy()

df_q3["is_pregnant"] = (df_q3["Pregnancies"] >= 1) + 0
df_q3.head(2)
lm = smf.ols("BMI ~ is_pregnant", df_q3).fit()
print(round(lm.tvalues[1], 3))
print(round(lm.pvalues[1],3))

print(ttest_ind(df_q3.loc[df_q3["is_pregnant"] == 0, "BMI"], df_q3.loc[df_q3["is_pregnant"] == 1, "BMI"]))
lm.summary()

-3.341
0.001
Ttest_indResult(statistic=3.3413391396804033, pvalue=0.0008741720747529957)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BMI   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     11.16
Date:                Fri, 27 Nov 2020   Prob (F-statistic):           0.000874
Time:                        13:09:27   Log-Likelihood:                -2669.5
No. Observations:                 768   AIC:                             5343.
Df Residuals:                     766   BIC:                             5352.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      34.2901      0.743     46.125      0.000      32.831      35.749
is_pregnant    -2.6857      0.804     -3.341      0.001      -4.264      -1.108
==============================================================================
Omnibus:                      101.731   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              413.512
Skew:                          -0.550   Prob(JB):                     1.61e-90
Kurtosis:                       6.422   Cond. No.                         5.08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Q4. 임신 횟수, 혈당, 혈압, 피부 두께로 BMI를 설명할 수 있는지 확인해보려 한다. BMI를 종속변수로 한 다중선형 회귀분석을 실시하고 유의미한 변수의 개수와 결정계수를 기술하시오.
 - 단, 이 분석은 70세 미만의 데이터만 사용한다.
 - 결측치는 모두 제거하고 분석을 실시한다.
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 1, 0.12)

In [57]:
df_sub = df[df["Age"] < 70]
df_sub.iloc[:, 1:-1] = df_sub.iloc[:, 1:-1].replace(0, np.nan)
df_sub = df_sub.dropna()
# df_sub = df_sub[df_sub.iloc[:,0:-1] != 0].dropna()

lm = smf.ols("BMI ~ Glucose + BloodPressure + SkinThickness", df_sub).fit()
print(", ".join(lm.pvalues[lm.pvalues >= 0.2].index))
print(", ".join(lm.pvalues[lm.pvalues >= 0.2].round(3).astype("str").values))

idx_1 = lm.pvalues[lm.pvalues >= 0.2].index
arr_1 = lm.pvalues[lm.pvalues >= 0.2].round(3)

params_1 = lm.params[idx_1]
print(", ".join(params_1.astype("str")))
lm.summary()

/home/rolroralra/venv/lib/python3.6/site-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BMI   R-squared:                       0.471
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     114.8
Date:                Fri, 27 Nov 2020   Prob (F-statistic):           3.41e-53
Time:                        13:34:20   Log-Likelihood:                -1192.2
No. Observations:                 391   AIC:                             2392.
Df Residuals:                     387   BIC:                             2408.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        13.5358      1.686      8.027      0.000      10.220      16.851
Glucose           0.0128      0.009      1.468      0.143      -0.004       0.030
BloodPressure     0.0840      0.022      3.876      0.000       0.041       0.127
SkinThickness     0.4141      0.026     16.116      0.000       0.364       0.465
==============================================================================
Omnibus:                       39.302   Durbin-Watson:                   1.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               64.276
Skew:                           0.641   Prob(JB):                     1.10e-14
Kurtosis:                       4.517   Cond. No.                         957.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [67]:
df_q4 = df.copy()
df_q4 = df_q4.loc[df_q4["Age"] < 70, ]
df_q4 = df_q4.iloc[:,1:-1].replace(0, np.nan).dropna()
df_q4

lm = smf.ols("BMI ~ Glucose + BloodPressure + SkinThickness", df_q4).fit()
lm.summary()
lm_q4 = lm.rsquared
print(lm_q4)


# print(", ".join(lm.pvalues[lm.pvalues >= 0.2].index))
# print(", ".join(lm.pvalues[lm.pvalues >= 0.2].round(3).astype("str").values))

# idx_1 = lm.pvalues[lm.pvalues >= 0.2].index
# arr_1 = lm.pvalues[lm.pvalues >= 0.2].round(3)

# params_1 = lm.params[idx_1]
# print(", ".join(params_1.astype("str")))

0.470941268953485


### Q5. 결측치 처리에 따른 효과를 알아보고자 한다. 4번에서 사용된 독립변수 중 결측치를 모두 해당 변수의 평균치로 치환하고 다중선형 회귀분석을 실시하여 얻은 결정계수와 4번에서 계산한 결정계수와의 차이의 절대값을 기술하시오.
 - 단, 이 분석은 70세 미만의 데이터만 사용한다.
 - 종속변수와 독립변수는 4번과 동일하게 한다.
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 0.12, 0.34)

In [69]:
df_q5 = df.copy()
df_q5 = df_q5.loc[df_q5["Age"] < 70,]

df_q5.iloc[:, 1:-1] = df_q5.iloc[:, 1:-1].replace(0, np.nan)
df_q5 = df_q5.fillna(df_q5.mean())
print(df_q5.head())
print()

lm = smf.ols("BMI ~ Glucose + BloodPressure + SkinThickness", df_q5).fit()
lm.summary()
lm_q5 = lm.rsquared
print(abs(lm_q5 - lm_q4))

   Pregnancies  Glucose  BloodPressure  SkinThickness     Insulin   BMI  \
0            6    148.0           72.0      35.000000  155.791349  33.6   
1            1     85.0           66.0      29.000000  155.791349  26.6   
2            8    183.0           64.0      29.166976  155.791349  23.3   
3            1     89.0           66.0      23.000000   94.000000  28.1   
4            0    137.0           40.0      35.000000  168.000000  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  

0.1321670018384805


In [60]:
df_sub = df[df["Age"] < 70]
# print(df_sub.describe())
df_sub = df_sub[df_sub != 0].fillna(df_sub.mean())
print(df_sub.head())

lm = smf.ols("BMI ~ Glucose + BloodPressure + SkinThickness", df_sub).fit()
lm.summary()

params_2 = lm.params[idx_1]


print(", ".join(params_2.round(3).astype("str")))
print(", ".join((params_1 - params_2).abs().round(3).astype("str")))
# print(", ".join(lm.pvalues[lm.pvalues >= 0.2].index))
# print(", ".join(lm.pvalues[lm.pvalues >= 0.2].round(3).astype("str").values))

   Pregnancies  Glucose  BloodPressure  SkinThickness     Insulin   BMI  \
0            6    148.0           72.0      35.000000  155.791349  33.6   
1            1     85.0           66.0      29.000000  155.791349  26.6   
2            8    183.0           64.0      29.166976  155.791349  23.3   
3            1     89.0           66.0      23.000000   94.000000  28.1   
4            0    137.0           40.0      35.000000  168.000000  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  

   Pregnancies  Glucose  BloodPressure  SkinThickness     Insulin   BMI  \
0     6.000000    148.0           72.0      35.000000   80.033987  33.6   
1     1.000000     85.0           66.0      29.000000   80.033987  26.6   
2     8.000000    183.0           64.0      20.550327   80.

### Q6. 분류모델의 성능을 비교하고자 한다. Outcome 변수를 독립변수로 하고 나머지 변수를 독립변수로 하고, 독립변수의 경우 결측치로 간주되는 것은 모두 제거한다. 이항 로지스틱 회귀분석과, 의사결정나무로 학습을 하고 그 정확도를 각각 기술하시오.
 - 학습은 최초 100개 데이터를 제외한 나머지로 실시한다.
 - 평가는 최초 100개 데이터로 실시한다.
 - 이항 로지스틱 회귀분석의 경우 유의미 하지 않은 변수는 제거한다.
 - 이항 로지스틱 회귀분석의 문턱값은 0.5로 설정한다.
 - 정확도는 반올림하여 소수점 셋째 자리까지 표기한다.
 
(정답 예시: 0.123, 0.456)

In [87]:
df_q6 = df.copy()

df_q6.iloc[:, 1:-1] = df_q6.iloc[:, 1:-1].replace(0, np.nan)
df_q6 = df_q6.dropna()

df_q6_train = df_q6.iloc[100:, ]
df_q6_test = df_q6.iloc[:100, ]
# df_q6_train.shape[0] + df_q6_test.shape[0] == df_q6.shape[0]

from statsmodels.api import Logit
from sklearn.tree import DecisionTreeClassifier

model_logit = Logit(df_q6_train["Outcome"], df_q6_train.drop("Outcome", axis=1)).fit()

model_logit.summary()

Optimization terminated successfully.
         Current function value: 0.538995
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Outcome   No. Observations:                  292
Model:                          Logit   Df Residuals:                      284
Method:                           MLE   Df Model:                            7
Date:                Fri, 27 Nov 2020   Pseudo R-squ.:                  0.1234
Time:                        13:47:11   Log-Likelihood:                -157.39
converged:                       True   LL-Null:                       -179.54
Covariance Type:            nonrobust   LLR p-value:                 1.859e-07
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Pregnancies                  0.1348      0.059      2.280      0.023       0.019       0.251
Glucose                      0.0220      0.006      3.897      0.000       0.011       0.033
BloodPressure               -0.0505      0.012     -4.192      0.000      -0.074      -0.027
SkinThickness                0.0401      0.018      2.241      0.025       0.005       0.075
Insulin                      0.0018      0.001      1.323      0.186      -0.001       0.005
BMI                         -0.0594      0.028     -2.137      0.033      -0.114      -0.005
DiabetesPedigreeFunction     0.4404      0.409      1.077      0.282      -0.361       1.242
Age                         -0.0037      0.019     -0.194      0.846      -0.042       0.034
============================================================================================
"""

In [106]:
pred_logit_prob = model_logit.predict(df_q6_test.drop("Outcome", axis = 1))
pred_logit = (pred_logit_prob > 0.5) + 0
print(round(accuracy_score(df_q6_test["Outcome"], pred_logit), 3))

0.66


In [91]:
model_tree = DecisionTreeClassifier().fit(X=df_q6_train.drop("Outcome", axis=1), y=df_q6_train["Outcome"])
pred_tree = model_tree.predict(df_q6_test.drop("Outcome", axis=1))
print(round(accuracy_score(df_q6_test["Outcome"], pred_tree), 3))

0.68


In [107]:
from sklearn.linear_model import LogisticRegression
# model_Logistic_sl = LogisticRegression()

In [122]:
df_q6 = df.copy()

df_q6.iloc[:, 1:-1] = df_q6.iloc[:, 1:-1].replace(0, np.nan)
df_q6 = df_q6.dropna()

from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(df_q6.drop("Outcome",axis=1), df_q6["Outcome"], test_size=100)
df_train, df_test = train_test_split(df, test_size=100)

import statsmodels.api as sm
from statsmodels.formula.api import logit
new_model = logit(f"Outcome ~ {' + '.join(df_q6.drop(['Outcome'], axis=1).columns)}", df_train).fit()
pred_new_logic = (new_model.predict(df_test.drop("Outcome", axis=1)) > 0.5) + 0
print(round(accuracy_score(df_test["Outcome"], pred_new_logic), 3))

# model = sm.Logit(df_q6["Outcome"].astype("int"), df_q6.drop("Outcome", axis=1))
# result = model.fit(f"Outcome ~ {' + '.join(df_q6.drop(['Outcome'], axis=1).columns)}")


392
Optimization terminated successfully.
         Current function value: 0.467077
         Iterations 6
0.79
